In [1]:
from zappai.zappai.di import (
    get_crop_optimizer_service,
    get_session_maker,
    get_cds_api,
    get_location_repository,
    get_past_climate_data_repository,
    get_future_climate_data_repository,
    get_climate_generative_model_repository,
    get_crop_repository,
    get_crop_yield_data_repository,
    get_crop_yield_model_service,
)
session_maker = get_session_maker()
cds_api = get_cds_api()
location_repository = get_location_repository()
past_climate_data_repository = get_past_climate_data_repository(
    cds_api=cds_api,
    location_repository=location_repository,
)
future_climate_data_repository = get_future_climate_data_repository(cds_api=cds_api)
climate_generative_model_repository = get_climate_generative_model_repository(
    location_repository=location_repository,
    past_climate_data_repository=past_climate_data_repository,
    future_climate_data_repository=future_climate_data_repository,
)
crop_repository = get_crop_repository()
crop_yield_data_repository = get_crop_yield_data_repository(
    crop_repository=crop_repository,
    location_repository=location_repository,
    past_climate_data_repository=past_climate_data_repository,
)
crop_yield_model_service = get_crop_yield_model_service(
    past_climate_data_repository=past_climate_data_repository,
    location_repository=location_repository,
    crop_yield_data_repository=crop_yield_data_repository,
    crop_repository=crop_repository,
)
crop_optimizer_service = get_crop_optimizer_service(
    past_climate_data_repository=past_climate_data_repository,
    location_repository=location_repository,
    climate_generative_model_repository=climate_generative_model_repository,
    crop_repository=crop_repository,
    future_climate_data_repository=future_climate_data_repository,
)

2024-07-27 12:20:58.693714: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-27 12:20:58.747337: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-27 12:20:58.998042: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-27 12:20:59.851098: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
async with session_maker() as session:
    location = await location_repository.get_location_by_country_and_name(
        session=session, country="Italy", name="Policoro"
    )

    if location is None:
        raise Exception()

    crop = await crop_repository.get_crop_by_name(session=session, name="maize")
    if crop is None:
        raise Exception()

# result = await crop_optimizer_service.get_best_crop_sowing_and_harvesting(crop_id=crop.id, location_id=location.id)
async with session_maker() as session:
    result = await crop_optimizer_service.get_best_crop_sowing_and_harvesting(
        session=session, crop_id=crop.id, location_id=location.id
    )

best_combination = result.best_combinations[-1]
print(f"Sowing: {best_combination.sowing_year}/{best_combination.sowing_month}")
print(f"Harvesting: {best_combination.harvest_year}/{best_combination.harvest_month}")
print(f"Duration: {best_combination.duration}")
print(f"Estimated yield per hectar: {best_combination.estimated_yield_per_hectar}")

Exception: 